In [ ]:
!git clone https://github.com/laxmimerit/dog-cat-full-dataset.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, ZeroPadding2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.optimizers import Adam

print(tf.__version__)

In [ ]:
test_data_dir = '/content/dog-cat-full-dataset/data/test'
train_data_dir = '/content/dog-cat-full-dataset/data/train'

In [ ]:
img_width = 32
img_height = 32
batch_size = 20

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(directory=train_data_dir,
                                              target_size = (img_width, img_height),
                                              classes = ['dogs', 'cats'],
                                              class_mode = 'binary',
                                              batch_size=batch_size)

In [ ]:
validation_generator = datagen.flow_from_directory(directory=test_data_dir,
                                                   target_size = (32, 32),
                                                   classes = ['dogs', 'cats'],
                                                   class_mode = 'binary',
                                                   batch_size = batch_size)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='he_uniform', input_shape = (img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(filters=256, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

In [ ]:
history = model.fit_generator(generator=train_generator, steps_per_epoch=len(train_generator), 
                              epochs = 10, validation_data=validation_generator, 
                              validation_steps=len(validation_generator), verbose = 1)

In [ ]:
from tensorflow.keras.preprocessing import image
from google.colab.patches import cv2_imshow


img_pred = image.load_img("/content/dog-cat-full-dataset/data/test/cats/cat.10060.jpg",target_size=(32,32))
img_pred = image.img_to_array(img_pred)
img_pred = np.expand_dims(img_pred,axis=0)
rslt = model.predict(img_pred)
print(rslt)
prediction = ""
if rslt[0][0] == 1:
  prediction = 'Cat'
else:
  prediction = 'Dog'
print('prediction',prediction)

path = r"/content/dog-cat-full-dataset/data/test/cats/cat.10060.jpg"
image = cv2.imread(path)
window_name = 'Image'
font = cv2.FONT_HERSHEY_COMPLEX
org = (150,50)
fontScale = 2
color = (255,0,0)
thickness = 2
image = cv2.putText(image,prediction,org,font,fontScale,color,thickness,cv2.LINE_AA)
cv2_imshow(image)